<a href="https://colab.research.google.com/github/cypbk/colab-copies/blob/main/Sydney_rental_map_webV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit
!pip install pyngrok 
!pip install tornado
# 2OH1jTDodz747RKclVG68FzdQrB_21FRifjjyGcXFh9hzLaFf # token
!./ngrok authtokens 2OH1jTDodz747RKclVG68FzdQrB_21FRifjjyGcXFh9hzLaFf

from pyngrok import ngrok 
#public_url = ngrok.connect(port='8501')
#public_url


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: ./ngrok: No such file or directory


In [ ]:
#from pyngrok import ngrok 
public_url = ngrok.connect(port='8501')
public_url

PyngrokNgrokHTTPError: ignored

In [ ]:
!pip install geopandas
!pip install descartes
!pip install mapclassify 
!pip install adjustText
!pip install streamlit-folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# backup for the first version
%%writefile streamlit_app.py 
import mapclassify
import plotly.express as px
# import pyproj
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
import folium
from folium import plugins
from folium.plugins import FastMarkerCluster
from folium.plugins import TimestampedGeoJson
import os 
from os.path import join
from google.colab import drive
import pandas as pd
import numpy as np
import json
import os 
import re
import requests
import geopandas as gpd
import adjustText as aT
import streamlit as st
from streamlit_folium import st_folium
import folium
from IPython.core.display import Markdown

data_link = '/content/drive/MyDrive/Rent-tables-December-2022-quarter.xlsx'
shape_path = '/content/drive/MyDrive/Non_abs_data/POA_2021_AUST_GDA94.shp'

data= pd.read_excel(data_link, sheet_name='Postcode', skiprows=range(8))

# rename columns
data.rename(columns = {'Postcode':'postcode',
                       'Dwelling Types':'types',
                       'Number of Bedrooms':'rooms',
                       'Median Weekly Rent for New Bonds\n$':'rent_median',
                       'Quarterly change in Median Weekly Rent':'quarter_change_rent',
                       'Annual change in Median Weekly Rent':'annual_change_rent'},
            inplace = True)

all_postcodes = list(set(data['postcode']))

lga_postcodes = {
    'Ashfield':[2045,2130,2131,2132,2133,2193],
    'Auburn':[2141,2143],
    'Bankstown':[2190, 2195, 2196, 2198, 2200],
    'Bass Hill':[2162,2197, 2199],
    'Berowra':[2080,2081,2082],
    'Blackheath Katoomba':[2780, 2782, 2785, 2786, 2790],
    'Blacktown':[2147, 2148, 2763, 2766,2767, 2768, 2769],
    'Botany':[2018, 2019, 2036],
    'Burwood':[2133,2134,2136],
    'Camden Wollondilly':[2570,2571,2572,2574],
    'Campbelltown':[2555, 2558, 2559, 2560,2563,2565,2566],
    'Canada Bay':[2046,2047],
    'Canterbury':[2191,2192,2193,2194,2206],
    'Cherrybrook':[2126,2155,2156,2157,2158,2159],
    'Concord':[2046,2137,2138],
    'East Hills':[2211,2212,2213,2214],
    'Fairfield':[2161,2163,2164,2165,2166],
    'Frenchs Forest':[2084,2085,2086,2087],
    'Hawkesbury River':[2083],
    'Hazelbrook':[2778,2779,2783,2784],
    'Holroyd':[2142, 2145, 2148, 2160],
    'Hornsby':[2076,2077,2079,2119,2120,2125],
    'Hunters Hill':[2110,2111],
    'Hurstville':[2208, 2209, 2210, 2220, 2221, 2222, 2223],
    'Kogarah':[2217, 2218, 2220, 2221, 2223],
    'Kuringai':[2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076],
    'Kurrajong Wisemans Ferry':[2758, 2775],
    'Lane Cove':[2066],
    'Leichhardt':[2038, 2039,2040,2041],
    'Liverpool':[2166, 2167,2168, 2170, 2171, 2172, 2173, 2174, 2175, 2176, 2177, 2178, 2179, 2564],
    'Manly Warringah':[2092, 2093, 2094, 2095, 2096, 2097, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108],
    'Marrickville':[2042, 2044, 2048, 2049, 2050, 2203, 2204],
    'Mascot':[2020],
    'Mosman':[2088, 2089, 2090],
    'Narellan':[2556, 2557, 2567, 2568, 2569, 2570, 2571, 2745],
    'Olympic Park':[2127, 2128],
    'Parramatta':[2115, 2116, 2117, 2118, 2142, 2145, 2146, 2150, 2151, 2152, 2153, 2154],
    'Penrith':[2745, 2747, 2749, 2750, 2751, 2752],
    'Randwick':[2031, 2032, 2033, 2034, 2035, 2036],
    'Rhodes':[2137, 2138],
    'Richmond Windsor':[2753, 2754, 2756, 2757, 2765, ],
    'Rockdale':[2205, 2206, 2207, 2216, 2217, 2219],
    'Ryde Eastwood':[2111, 2112, 2113, 2114, 2121, 2122],
    'Springwood':[2776, 2777, ],
    'St Marys':[2747, 2748, 2759, 2760, 2761, 2762, 2763, 2766, 2770],
    'Strathfield':[2129, 2135, 2136, 2140],
    'Sutherland':[2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2508],
    'Sydney Airport':[2020],
    'Sydney City':[2000, 2007, 2008, 2009, 2010, 2011, 2012, 2015, 2016, 2017, 2021, 2037, 2043],
    'Sydney Lower North Shore':[2055, 2057, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2067, 2068, 2069],
    'Waverley':[2027, 2028, 2029, 2030],
    'Woollahra':[2021, 2022, 2023, 2024, 2025, 2026] 
    }

post_groups = list(lga_postcodes.keys())

postcode_Canterbury = [2191,2192,2193,2194,2206] # deafault values of selected postcodes
num_rooms = list(set(data['rooms']))
dwelling_types = list(set(data['types']))

st.set_page_config(page_title="Sydney rent map", layout="wide")

# Initialization
if 'default_postcodes' not in st.session_state:
  st.session_state['default_postcodes'] = postcode_Canterbury #postcode_Canterbury 

if 'default_regions' not in st.session_state:
  st.session_state['default_regions'] = ['Canterbury'] #postcode_Canterbury 

st.sidebar.header('Set postcodes')
selected_postcodes = st.sidebar.multiselect(
    label = 'Choose postcodes you want to know the median rents in those area:',
    options = all_postcodes,
    key = 'selected_postcodes', 
    default = st.session_state['default_postcodes'])

def lga_group_callback():    
    postcodes_all = []
    #st.session_state['default_postcodes'] = []
    chosen_groups = st.session_state['default_regions'] 
    for group in chosen_groups:
      postcodes = lga_postcodes[group] 
      postcodes_all.extend(postcodes)
      #st.session_state['default_postcodes'].extend(postcodes)
    
    st.session_state['default_postcodes'] = postcodes_all
 
st.sidebar.header('Or select postcodes based on specific regions:')
selected_regions = st.sidebar.multiselect(
    label = 'Choose areas you are interested in:',
    options = post_groups,
    key = 'region',
    default = st.session_state['default_regions']
    )

st.session_state['default_regions'] = selected_regions
#st.write(st.session_state['default_regions'])


if 'default_postcodes' not in st.session_state:
  st.session_state['default_postcodes'] = postcode_Canterbury #postcode_Canterbury 

postcodes_all = []
chosen_groups = st.session_state['default_regions'] #st.session_state['default_regions'] 
for group in chosen_groups:
  postcodes = lga_postcodes[group] 
  postcodes_all.extend(postcodes)
    
st.session_state['default_postcodes'] = postcodes_all

st.sidebar.header('Dwelling types')
d_type = st.sidebar.selectbox(label = 'What kind of accommodation you are looking for?', 
                options = dwelling_types
                )

st.sidebar.header('Number of bedrooms')
n_room = st.sidebar.selectbox(label = 'How many rooms are needed?', 
                options = num_rooms)

@st.cache_data
def data_filter(data, postcode, types, rooms):
  #df[df['post'].isin([1,3,5])]
  filter_postcodes = data['postcode'].isin(postcode)
  filter_type = data['types']==types #data['types'].isin(types) #data['types']==types 
  filter_room = data['rooms']==rooms #data['rooms'].isin(rooms) #data['rooms']==rooms #	
  all_filters = filter_postcodes & filter_type & filter_room 
  selected_data = data[all_filters]

  df_rent = selected_data[['postcode', 'types', 'rooms','rent_median','quarter_change_rent','annual_change_rent']]
  df_rent['rent_median'] = pd.to_numeric(df_rent['rent_median'], errors='coerce')
  df_rent['quarter_change_rent'] = pd.to_numeric(df_rent['quarter_change_rent'], errors='coerce')
  df_rent['annual_change_rent'] = pd.to_numeric(df_rent['annual_change_rent'], errors='coerce')

  return df_rent

@st.cache_data
def get_map_data(df_rent, shape_path):
  region = gpd.read_file(shape_path)
  region['POA_CODE21'] = pd.to_numeric(region['POA_CODE21'], errors='coerce')
  region.rename(columns = {'POA_CODE21':'postcode'}, inplace = True)
  region.drop(region.columns.difference(['postcode', 'geometry']), axis=1, inplace=True)
  
  # Merge gpd dataframe with pandas dataframe
  region_map = region.merge(df_rent, on='postcode')
  e_col = region_map.pop('geometry')
  region_map.insert(1, 'geometry', e_col)
  #region_map = region_map[region_map['postcode'].isin(selected_regions)]
  return region_map

df_rent = data_filter(data, selected_postcodes, d_type, n_room)
df_map = get_map_data(df_rent, shape_path)


# full map
m = df_map.explore(column='rent_median',  # make choropleth based on "BoroName" column
     cmap ='Purples',
     #tooltip="rent_median", 
     legend=True, # show legend
     k=10, # use 10 bins
     legend_kwds=dict(colorbar=True), # do not use colorbar
     )

# Heatmap
fig = px.choropleth(df_map,
     geojson=df_map.geometry,
     locations=df_map.index,
     hover_name = 'postcode',
     color='rent_median', 
     color_continuous_scale = 'Purples'
     )
fig.update_geos(fitbounds="locations", visible=False)

st.title('Sydney rent map')
st.header("A rent map based on postcodes of Sydney resgion")
st.markdown('This page is for visualization of the median rent of the great Sydney area. I tried to make this map because only the visualization of median rent based on local government areas (which looks really beautiful) can be found at the the office website of NSW. But I am trying to look a bit more closer.')
st.markdown('Dwellings rental data by LGA/postcodes for December 2022 quarter can be downloaded from https://www.facs.nsw.gov.au/resources/statistics/rent-and-sales/dashboard .')

tab_full, tab_heatmap = st.tabs(['With actual map', "Heatmap only"])

with tab_full:
    st.header('Map with geographic information')
    final_map = st_folium(m)
with tab_heatmap:
    st.header('Heatmap of rents for choosing areas')
    st.write(fig)

st.header("Reference")
st.markdown('There were already similar analyses conducted by Tariq Munir and walthersy@Github (and his colleadgues). Codes and instructions could be found via the following links:')
st.markdown('1. https://towardsdatascience.com/exploring-greater-sydney-suburbs-f2bf1562988e  ')
st.markdown('2. https://github.com/Timun01/IBM-Capstone-Project')
st.markdown('3. https://github.com/walthersy/NSW-house-price')

st.markdown('Many thnks to their kindly sharing.')

Overwriting streamlit_app.py


In [ ]:
!streamlit run streamlit_app.py & npx localtunnel --port 8501

npx: installed 22 in 6.398s


your url is: https://tall-needles-teach-34-133-193-49.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.133.193.49:8501

/content/streamlit_app.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rent['rent_median'] = pd.to_numeric(df_rent['rent_median'], errors='coerce')
/content/streamlit_app.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rent['quarter_change_rent'] = pd.to_numeric(